# 🚀 **Fine-Tuning Falcon-7B-Instruct with LoRA on a Mental Health Conversational Dataset**

* This notebook demonstrates the fine-tuning of the **Falcon-7B-Instruct Sharded Model** on a mental health conversational dataset curated by **heliosbrahma**.  

* Both the model and dataset are available on **Hugging Face**, and links are provided in the notebook.

---

## 📌 **1. Setup Dependencies and Imports**
First, we will install the required dependencies, configure the Hugging Face login, and prepare the environment for model training.

In [ ]:
#all installs
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install huggingface_hub

#all imports
import torch
import time
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from transformers import TrainingArguments
from trl import SFTTrainer

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

---

## 🔗 **2. Connect to Hugging Face Hub**
We'll authenticate with Hugging Face to access the datasets and models.

In [ ]:
!huggingface-cli login

In [ ]:
notebook_login()

---

## 📥 **3. Load the Dataset**
We will load the mental health conversational dataset directly from **Hugging Face Datasets**.

In [ ]:
dataset_name = "heliosbrahma/mental_health_chatbot_dataset"
data = load_dataset(dataset_name)
data

---

## 💡 **4. Model Loading and Configuration**
We are using the **Falcon-7B-Instruct Sharded Model**.  
> **Sharding**: This technique splits the model into smaller chunks, allowing for parallel processing across multiple devices—ideal for memory-constrained environments.  

In [ ]:
model_name = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False

---

## 🔄 **5. Load the Tokenizer**
We load the tokenizer for text processing, which is essential for converting text data into model-compatible formats.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

---

## 🔧 **6. Configure LoRA (Low-Rank Adaptation)**
We set up the **LoRA configuration** for parameter-efficient fine-tuning.  
Below are the main parameters:  

- **lora_rank**: Defines the low-rank dimensions. Smaller ranks result in fewer trainable parameters (e.g., 512x32 and 32x512 in adapter layers).  
- **lora_alpha**: Scaling factor that adjusts the magnitude of LoRA layers.  
- **lora_dropout**: Dropout rate to prevent overfitting during training.  
- **bias**: Specifies if bias parameters should be trained (`none`, `all`, or `lora_only`).  
- **task_type**: Since this is a text generation task, it is set to `CAUSAL_LM`.  
- **target_modules**: Specifies the model components (like attention blocks) where LoRA updates are applied.

In [ ]:
model = prepare_model_for_kbit_training(model)

lora_alpha = 32 #16
lora_dropout = 0.05 #0.1
lora_rank = 32 #64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

peft_model = get_peft_model(model, peft_config)

---

## 🏋️ **7. Initialize the Trainer**
We initialize the **SFTT Trainer** with appropriate parameters for fine-tuning.  

In [ ]:
output_dir = "falcon7binstruct_mentalhealthmodel_oct23"
per_device_train_batch_size = 16 #4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 180 #100 #500
warmup_ratio = 0.03
lr_scheduler_type = "cosine" #"constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True
)

---

## ⚙️ **8. Configure Training Arguments (Passing arguments to the SFTT trainer)**
We provide necessary training arguments to the trainer for optimal performance and stability.

In [ ]:
max_seq_length = 256

trainer = SFTTrainer(
    model=peft_model,
    train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

### For more stable training

In [ ]:
# upcasting the layer norms in torch.bfloat16 for more stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.bfloat16)

---

## 🚀 **9. Model Training**
Finally, we train the model using the defined configurations.  
You can monitor your training in **Weights & Biases** for real-time metrics.  

> 💡 **Tip**: If you are using **Google Colab**, prevent disconnection with the following snippet:  
> Open the console (`Ctrl+Shift+I` → Console tab) and paste:  
```javascript
function ClickConnect(){
    console.log("Working");
    document.querySelector("colab-toolbar-button").click();
}
setInterval(ClickConnect, 60000);
```

In [ ]:
start = time.time()

In [ ]:
peft_model.config.use_cache = False
trainer.train()

In [ ]:
end=time.time()

In [ ]:
time_taken=end-start
print(time_taken)

---

## 💾 **10. Save the Fine-Tuned Model**
Save the fine-tuned model to your local or cloud storage for future use.

In [ ]:
#trainer.save() # if you want to save your model locally

---

## ☁️ **11. Push to Hugging Face Hub**
Upload the trained model to **Hugging Face Hub** to share and reuse.

In [ ]:
trainer.push_to_hub()

> 📊 **Check Training Metrics**: Visit your **Weights & Biases** account for detailed insights into training performance.

---

## 🔍 **12. Inference**
For inference, you will need to:  
1. **Clear memory or restart the kernel:** For inference from your original model and the fine-tuned model, you need to clean your memory or restart your kernel. Remember to push the fine-tuned model to Hugging Face so that you can pull your new model.  
2. **Load both models:** Load your original model as before, then load your fine-tuned model. This wraps the base model with your PEFT model.


In [ ]:
# Loading PEFT model
PEFT_MODEL = "Srishy/falcon7binstruct_mentalhealthmodel_oct23"
config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

---

## 💡 **13. Model Response Comparison**
Finally, let's see how the responses differ between the **original model** and the **fine-tuned model** on the same input prompts.

In [ ]:
# Generate responses from both orignal model and fine-tuned model
def get_response(question):
  prompt = f"""
  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: {question}

  ###Response:

  """

  encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")
  outputs = model.generate(input_ids=encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = tokenizer.eos_token_id, attention_mask = encoding.attention_mask, \
                                                                                                                     temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

  #print(dashline)
  print(f'Response from original falcon_7b_instruct_sharded:\n{text_output}')

  print("*******************************************************")

  peft_encoding = peft_tokenizer(prompt, return_tensors="pt").to("cuda:0")
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                      eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                      temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

  print(f'Response from fine-tuned falcon_7b_instruct_sharded:\n{peft_text_output}')


In [ ]:
get_response("Are there cures for mental health problems?")

In [ ]:
get_response("What’s the difference between psychotherapy and counselling?")

In [ ]:
get_response("What happens in a therapy session?")

In [ ]:
get_response("Are neurofeedback and biofeedback the same thing?")

In [ ]:
get_response("What is insomnia disorder?")

---

### ✅ **Conclusion**

You have successfully fine-tuned the **Falcon-7B-Instruct** model on a mental health conversational dataset using **LoRA**. The fine-tuned model is now ready for deployment or further experimentation.
